In [1]:
# Install necessary libraries
%pip install "pybamm[plot,cite]" -q

import pybamm
import matplotlib.pyplot as plt

# Define the battery model
model = pybamm.lithium_ion.DFN(
    {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "SEI on cracks": "true",
        "loss of active material": "stress-driven",
        "calculate discharge energy": "true",
    }
)

# Set the model parameters
param = pybamm.ParameterValues("OKane2022")
var_pts = {
    "x_n": 5,  # Negative electrode
    "x_s": 5,  # Separator
    "x_p": 5,  # Positive electrode
    "r_n": 30,  # Negative electrode particle
    "r_p": 30,  # Positive electrode particle
}

# Set the target number of cycles
target_cycle = 500

print(f"Calculating cycle {target_cycle} ...")

# Define the experiment protocol
exp_k = pybamm.Experiment(
    [
        "Hold at 4.2 V until C/100 (5 minute period)",
        "Rest for 4 hours (5 minute period)",
        "Discharge at 0.1C until 2.5 V (5 minute period)",
        "Charge at 0.3C until 4.2 V (5 minute period)",
        "Hold at 4.2 V until C/100 (5 minute period)",
    ]
    + [
        (
            "Discharge at 1C until 2.5 V",
            "Charge at 2C until 4.2 V (5 minute period)",
            "Hold at 4.2 V until C/100 (5 minute period)",
        )
    ]
    * target_cycle
    + ["Discharge at 0.1C until 2.5 V (5 minute period)"],
)

# Run the simulation
sim_k = pybamm.Simulation(model, parameter_values=param, experiment=exp_k, var_pts=var_pts)
sol_k = sim_k.solve()

# Extract LLI loss (in A.h) and calculate remaining capacity
Q_LLI_final_k = sol_k["Total lithium lost [mol]"].entries[-1] * 96485.3 / 3600  # Convert units
remaining_capacity_k = (5 - Q_LLI_final_k) / 5 * 100  # Calculate remaining capacity percentage

print(f"At k={target_cycle}, the remaining capacity is: {remaining_capacity_k:.2f}%")



Note: you may need to restart the kernel to use updated packages.
Calculating cycle 500 ...


At t = 429.689 and h = 1.24737e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 189.689 and h = 9.66951e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 428.881 and h = 9.84615e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 188.881 and h = 1.53332e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 428.316 and h = 7.14908e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 188.316 and h = 8.72162e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 324.035 and h = 2.6291e-10, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 187.783 and h = 1.0086e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 427.276 and h = 8.52486e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 187.276 and h = 1.07352e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At

At k=500, the remaining capacity is: 96.73%
